In [10]:
import numpy as np
import pandas as pd

import pyro
import pyro.distributions as dist
from pyro.infer import SVI, Trace_ELBO, MCMC, NUTS, HMC

import torch
from torch.distributions import constraints

import matplotlib.pyplot as plt
import seaborn as sns

import graphviz
import utils

import mph
from multiprocessing import Process
from multiprocessing import Queue
from multiprocessing import cpu_count
from queue import Empty


In [15]:
def multiprocessInit():
    jobs = Queue()
    freq = [100, 200]
    for f in freq:
        jobs.put(f)
    
    results = Queue()
    processes = []
    workers = cpu_count()
    for _ in range(workers):
        process = Process(target=worker, args=(jobs, results))
        processes.append(process)
        process.start()

def worker(jobs, results):
    client = mph.start(cores=1)
    model = client.load("../comsol/beam.mph")
    while True:
        try:
            d = jobs.get(block=False)
        except Empty:
            break
        print(d)
        model.parameter('freq', str(d)+' [Hz]')
        model.solve('study 3')
        Y = abs(model.evaluate('comp1.point1'))
        results.put((d, Y))

In [16]:
multiprocessInit()

Traceback (most recent call last):
  File "<string>", line 1, in <module>
  File "/opt/miniconda3/lib/python3.9/multiprocessing/spawn.py", line 116, in spawn_main
Traceback (most recent call last):
  File "<string>", line 1, in <module>
  File "/opt/miniconda3/lib/python3.9/multiprocessing/spawn.py", line 116, in spawn_main
    exitcode = _main(fd, parent_sentinel)
  File "/opt/miniconda3/lib/python3.9/multiprocessing/spawn.py", line 126, in _main
    self = reduction.pickle.load(from_parent)
AttributeError: Can't get attribute 'worker' on <module '__main__' (built-in)>
Traceback (most recent call last):
  File "<string>", line 1, in <module>
  File "/opt/miniconda3/lib/python3.9/multiprocessing/spawn.py", line 116, in spawn_main
    exitcode = _main(fd, parent_sentinel)
  File "/opt/miniconda3/lib/python3.9/multiprocessing/spawn.py", line 126, in _main
    self = reduction.pickle.load(from_parent)
AttributeError: Can't get attribute 'worker' on <module '__main__' (built-in)>
    exitc